# Problem 1

(a) Find the covariance matrix of $X_T$

$$\begin{aligned}
    d \log{S_t} &= \frac{1}{S_t} dS_t + \frac{-1}{2S_t^2} dS_t^2 \\
    &= \frac{1}{S_t} (rS_tdt + \sigma S_t dW_t) + \frac{-1}{2S_t^2} (\sigma^2 S_t^2 dt) \\
    &= (r-\frac{\sigma^2}{2})dt + \sigma dW_t \\
    \\
    \begin{bmatrix} X_t^{[1]} \\ X_t^{[2]} \end{bmatrix} &= \begin{bmatrix} \log S_t^{[1]} + (r-\frac{\sigma_{[1]}^2}{2})t\\ \log S_t^{[2]} + (r-\frac{\sigma_{[2]}^2}{2})t \end{bmatrix} + \begin{bmatrix} \sigma_{[1]}^2 & 0 \\ 0 & \sigma_{[2]}^2 \end{bmatrix} \begin{bmatrix} W_t^{[1]} \\ W_t^{[2]} \end{bmatrix} \\
    Cov(\begin{bmatrix} X_t^{[1]} \\ X_t^{[2]} \end{bmatrix}) &= \mathbb{E} [\Sigma W_T W_T^T \Sigma^T] \\
    &= \begin{bmatrix} 0.3 & 0 \\ 0 & 0.2 \end{bmatrix} \begin{bmatrix} 1T & 0.8T \\ 0.8T & 1T \end{bmatrix} \begin{bmatrix} 0.3 & 0 \\ 0 & 0.2 \end{bmatrix} \\
    &= \begin{bmatrix} 0.09T & 0.048T \\ 0.048T & 0.04T \end{bmatrix}
\end{aligned}$$

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
class Dynamics:
    pass

In [3]:
hw6p1dynamics=Dynamics()
hw6p1dynamics.S0 = np.array([100,110])
hw6p1dynamics.r = 0.05
hw6p1dynamics.correlations = np.array([[1, 0.8], [0.8, 1]])  #You fill this in with a 2x2 correlation matrix
hw6p1dynamics.sigma = np.diag([0.3, 0.2])

In [4]:
class Contract:
    pass

In [5]:
hw6p1contract=Contract() 
hw6p1contract.K = 110 
hw6p1contract.T = 1.0 
hw6p1contract.weights = np.array([1/2, 1/2])

In [6]:
class MC:
    pass

In [7]:
hw6p1MC=MC()
hw6p1MC.M = 10000  # Number of paths.  
hw6p1MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [8]:
M, S0 = hw6p1MC.M, hw6p1dynamics.S0
H = hw6p1dynamics.correlations

In [9]:
np.append(np.random.randn(M, len(S0)), np.random.randn(M, len(S0)), axis=0).shape

(20000, 2)

In [10]:
np.array([0.5, 0.5]).dot((np.ones((M, len(S0))) * S0).T)

array([105., 105., 105., ..., 105., 105., 105.])

In [11]:
def pricer_callonbasket_GBM_MC(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    K, T, weights = contract.K, contract.T, contract.weights
    S0, r, corr, sigma = dynamics.S0, dynamics.r, dynamics.correlations, dynamics.sigma
    M, antithetic, control = MC.M, MC.antithetic, MC.control
    
    if antithetic and control:
        raise ValueError("At most one of antithetic and control may be True.")
    
    # You complete the coding of this function
    # You are not required to support the case where MC.control = MC.antithetic = True
    # (simultaneous use of control variate and antithetic)
    # But you are required to support the other 3 possible settings of MC.antithetic/MC.control 
    # namely False/False, True/False, False/True.
    # (ordinary MC, antithetic without control, control without antithetic)
    def BScallPrice(sigma,S,rGrow,r,K,T):
        F=S*np.exp(rGrow*T)
        sd = sigma*np.sqrt(T)
        d1 = np.log(F/K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
    
    DEBUG=False
    
    L = np.linalg.cholesky(corr)
    X_0 = np.ones((M, len(S0)))
    X_0 = X_0 * np.log(S0)
    X_0 = X_0.T
    sigma_mat = (np.ones((M, len(S0))) * np.diag(sigma)).T
    
    if DEBUG: print(sigma_mat**2/2)
    
    dW_T = np.random.randn(len(S0), M) * np.sqrt(T)
    X_T = X_0 + (r-sigma_mat**2/2)*T + sigma.dot(L.dot(dW_T))
    
    if DEBUG: print(X_0, '\n', X_T)
    
    S_T = np.exp(X_T)
    H_T = weights.dot(S_T)
    C_T = np.exp(-r*T) * np.maximum(H_T-K, 0)
    
    if antithetic:
        X_Tneg = X_0 + (r-sigma_mat**2/2)*T - sigma.dot(L.dot(dW_T))
        S_Tneg = np.exp(X_Tneg)
        H_Tneg = weights.dot(S_Tneg)
        C_Tneg = np.exp(-r*T) * np.maximum(H_Tneg-K, 0)
        C_T = (C_T + C_Tneg) / 2
    
    if DEBUG: print(S_T, '\n', H_T)
    
    call_price = np.mean(C_T)
    standard_error = np.std(C_T, ddof=1) / np.sqrt(M)
    
    if control:
        sig = np.sqrt(sigma[0,0]**2 + sigma[1,1]**2 + 2*sigma[0,0]*sigma[1,1]*corr[0,1])/2
        S = (S0[0]*S0[1])**0.5
        rGrow = r + (2*sigma[0,0]*sigma[1,1]*corr[0,1] - sigma[0,0]**2 - sigma[1,1]**2)/8
        
        C_BS = BScallPrice(sig,S,rGrow,r,K,T)
        if DEBUG: print(sig, S, rGrow, C_BS)
        
        if DEBUG: print(X_T, '\n', np.mean(X_T, axis=0))
        G_Tcon = np.exp(np.mean(X_T, axis=0))
        C_Tcon = np.exp(-r*T) * np.maximum(G_Tcon-K, 0)
        
        cov = np.cov(C_T, C_Tcon)
        beta = cov[0,1] / cov[1,1]
        Y_con = C_T + beta*(C_BS - C_Tcon)
        call_price = np.mean(Y_con)
        standard_error = np.std(Y_con, ddof=1) / np.sqrt(M)
    
    return(call_price, standard_error)
        

In [12]:
hw6p1MC.antithetic = False  
hw6p1MC.control = False 
(call_price_ordinary, std_err_ordinary) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_ordinary, std_err_ordinary)

9.64432733712375 0.16404896020445345


In [13]:
hw6p1MC.antithetic = True  
hw6p1MC.control = False 
(call_price_AV, std_err_AV) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_AV, std_err_AV)

9.8252633692739 0.09350558499335318


(d)

$$\begin{aligned}
    \log G_t &= \frac{1}{2} \bigg(\log S_0^{[1]} + (r-\frac{\sigma_{[1]}^2}{2})t + \sigma_{[1]} W_t^{[1]} + \log S_0^{[2]} + (r-\frac{\sigma_{[2]}^2}{2})t + \sigma_{[2]} W_t^{[2]} \bigg) \\
    &= \frac{1}{2} \bigg(\log \big(S_0^{[1]}  S_0^{[2]} \big) + \big(2 * r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{2} \big)t + \sigma_{[1]} W_t^{[1]} + \sigma_{[2]} W_t^{[2]} \bigg) \\
    \mathbb{E} \big[ \log G_t \big] &= \frac{1}{2} \bigg(\log \big(S_0^{[1]}  S_0^{[2]} \big) + \big(2 * r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{2} \big)t \bigg) \\
    Var \big[ \log G_t \big] &= Var \bigg[ \frac{1}{2} \big(\log S_t^{[1]} + \log S_t^{[2]} \big) \bigg] \\
    &= \frac{1}{4} Var \bigg[ \log S_t^{[1]} + \log S_t^{[2]} \bigg] \\
    &= \frac{1}{4} \bigg( Var \big[ \log S_t^{[1]} \big] + Var \big[ \log S_t^{[2]} \big] + 2Cov \big[ S_t^{[1]}, S_t^{[2]} \big] \bigg) \\
    &= \frac{1}{4} \bigg( \sigma_{[1]}^2 + \sigma_{[2]}^2 + 2 \sigma_{[1]} \sigma_{[2]} \rho_{[1],[2]} \bigg) t \\
\end{aligned}$$

(e)

$$\begin{aligned}
    C^G =C^{BS}((S_0^{[1]}S_0^{[2]})^{\frac{1}{2}},0,K,T,r + \frac{ 2 \sigma_{[1]} \sigma_{[2]} \rho_{[1],[2]} - \sigma_{[1]}^2 - \sigma_{[2]}^2}{8},r,\frac{\sqrt{\sigma_{[1]}^2 + \sigma_{[2]}^2 + 2 \sigma_{[1]} \sigma_{[2]} \rho_{[1],[2]}}}{2} )
\end{aligned}$$

In [14]:
hw6p1MC.antithetic = False
hw6p1MC.control = True 
(call_price_CV, std_err_CV) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_CV, std_err_CV)

9.995159317013883 0.004409052033213148


## Problem 2

In [15]:
hw6p2dynamics=Dynamics()
hw6p2dynamics.sigma = 0.2
hw6p2dynamics.S0 = 100
hw6p2dynamics.r = 0.02

In [16]:
hw6p2contract=Contract()
hw6p2contract.K = 150
hw6p2contract.T = 1

In [17]:
hw6p2MC=MC()
hw6p2MC.M = 100000  # Number of paths
hw6p2MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [18]:
def pricer_call_GBM_MCwithdriftchange(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    K, T = contract.K, contract.T
    S0, r, sigma = dynamics.S0, dynamics.r, dynamics.sigma
    M, lamb = MC.M, MC.lamb
    
    # You complete the coding of this function
    
    W_1 = np.random.randn(M)
    X_T = np.log(S0) + (r-sigma**2/2)*T + sigma*np.sqrt(T)*(W_1 + lamb*T)
    S_T = np.exp(X_T)
    Y_T = np.exp(-r*T - lamb*W_1*np.sqrt(T) - 0.5*T*lamb**2) * np.maximum(S_T-K, 0)
    
    call_price = np.mean(Y_T)
    standard_error = np.std(Y_T, ddof=1) / np.sqrt(M)
        
    return(call_price, standard_error)
        

In [19]:
hw6p2MC.lamb = 0  # Zero drift adjustment gives ordinary MC
(call_price_ordinary, std_err_ordinary) =  pricer_call_GBM_MCwithdriftchange(hw6p2contract,hw6p2dynamics,hw6p2MC)
print(call_price_ordinary, std_err_ordinary)

0.24852756686116406 0.007435856213152823


(b)

$$\begin{aligned}
    d\tilde{W}_t &= dW_t^* + \lambda dt \\
    dS_t &= r S_t dt + \sigma S_t (dW_t^* + \lambda t) \\
    &= (r+\sigma \lambda)S_t dt + \sigma S_t dW_t^* \\
    d \log S_t &= (r+\sigma \lambda - \frac{\sigma^2}{2})dt + \sigma dW_t^* \\
    \mathbb{E}^* [S_t] &= S_0 e^{(r+\sigma \lambda)t} \\
    \lambda &= \frac{\log (\frac{\mathbb{E}^* [S_t]}{S_0}) / t - r}{\sigma}
\end{aligned}$$

In [20]:
ES_T = 165
S_0 = hw6p2dynamics.S0
r = hw6p2dynamics.r
sigma = hw6p2dynamics.sigma
T = hw6p2contract.T

hw6p2MC.lamb = (np.log(ES_T/S_0)/T - r) / sigma  # Fill this in with the lambda that you compute in (b) 
(call_price_importsamp, std_err_importsamp) =  pricer_call_GBM_MCwithdriftchange(hw6p2contract,hw6p2dynamics,hw6p2MC)
print(call_price_importsamp, std_err_importsamp)

0.249898640517768 0.0007761286383869399
